# Data Prep

Loads the output of a [transfermark-webscrapper](https://github.com/dcaribou/transfermarkt-scraper) run, and a applies a series of transformations to produce a file that is validated and more friendly for perfoming analyisis. Some of these transformations are

* Creating handy ID columns
* Renaming fileds to comply with naming convention
* Parsing raw values into their own columns

## Load
Input to the data prep process is excepted to be the output of the [transfermark-webscrapper](https://github.com/dcaribou/transfermarkt-scraper). I. e., a file with JSON lines with one line per player containing all player appearances on a game up to the date the scraper was run.

Later on we will use [papermill](https://papermill.readthedocs.io/en/latest/usage-parameterize.html) to run the notebook from a dvc pipeline. The next cells define [notebook-wide parameters](https://papermill.readthedocs.io/en/latest/usage-parameterize.html) than can be overridded via papermill arguments.

In [1]:
raw_file = 'data/raw/appearances.json'
prep_file = 'data/prep/appearances.csv'

In [2]:
raw_file = f"../{raw_file}"
prep_file = f"../{prep_file}"

In [3]:
import pandas as pd

raw = pd.read_json(
  raw_file,
  lines=True,
  convert_dates=True,
  orient={'index','date'}
)

raw.head()

,confederation,domestic_competition,stats_competition,current_team,player_name,stats
0,europa,ES1,ES1,athletic-bilbao,ander-capa,"[{'matchday': '1', 'date': '2020-09-12', 'home..."
1,europa,ES1,ES1,athletic-bilbao,dani-garcia,"[{'matchday': '1', 'date': '2020-09-12', 'home..."
2,europa,ES1,ES1,athletic-bilbao,inaki-williams,"[{'matchday': '1', 'date': '2020-09-12', 'home..."
3,europa,ES1,ES1,athletic-bilbao,unai-lopez,"[{'matchday': '1', 'date': '2020-09-12', 'home..."
4,europa,ES1,ES1,athletic-bilbao,oihan-sancet,"[{'matchday': '1', 'date': '2020-09-12', 'home..."


## Prep
The prep phase applies a series of transformations on the raw data frame that we loaded above

In [4]:
from prep_lib import *

### Flatten
Firstly, we need to explode the data frame to have one row per player appearance, rather than one row per player

In [5]:
raw_flat = flatten(raw, ['stats'])
raw_flat.head()

,matchday,date,home_team,away_team,result,pos,goals,assists,own_goals,yellow_cards,second_yellow_cards,red_cards,substitutions_on,substitutions_off,minutes_played,confederation,domestic_competition,stats_competition,current_team,player_name
0,1,2020-09-12,fc-granada,athletic-bilbao,2:0,RB,0,0,0,0,0,0,0,56',56,europa,ES1,ES1,athletic-bilbao,ander-capa
1,3,2020-09-27,sd-eibar,athletic-bilbao,1:2,RB,0,1,0,0,0,0,0,0,90,europa,ES1,ES1,athletic-bilbao,ander-capa
2,4,2020-10-01,athletic-bilbao,cadiz-cf,0:1,RB,0,0,0,0,0,0,0,86',86,europa,ES1,ES1,athletic-bilbao,ander-capa
3,5,2020-10-04,deportivo-alaves,athletic-bilbao,1:0,RB,0,0,0,0,0,0,0,0,90,europa,ES1,ES1,athletic-bilbao,ander-capa
4,6,2020-10-18,athletic-bilbao,ud-levante,2:0,RB,0,0,0,44',0,0,0,0,90,europa,ES1,ES1,athletic-bilbao,ander-capa


### Rename
Modify the names of the input columns to make them consisent with a naming convention

In [6]:
mappings = {
    'matchday': 'round',
    'home_team': 'home_club_name',
    'away_team': 'away_club_name',
    'current_team': 'player_club_name',
    'pos': 'player_position',
    'confederation': 'club_confederation',
    'domestic_competition': 'club_domestic_competition',
    'stats_competition': 'competition'
}

with_renamed_columns = renames(raw_flat, mappings)
with_renamed_columns.columns

Index(['round', 'date', 'home_club_name', 'away_club_name', 'result',
       'player_position', 'goals', 'assists', 'own_goals', 'yellow_cards',
       'second_yellow_cards', 'red_cards', 'substitutions_on',
       'substitutions_off', 'minutes_played', 'club_confederation',
       'club_domestic_competition', 'competition', 'player_club_name',
       'player_name'],
      dtype='object')

### Update
- [x] Convert `goals`, `assists`, `own_goals` and `date` to the appropriate types
- [x] Revamp `yellow_cards` and `red_cards`. `second_yellows` column is not needed
- [ ] Club name prettifying. _FC Watford_ instead of _fc-watford_
- [ ] Player name prettifying. _Adam Masina_ instead of _adam-masina_
- [ ] Use longer names for `position` instead of the chryptic 'LB', etc. (use 'filter by position' [here](https://www.transfermarkt.co.uk/diogo-jota/leistungsdatendetails/spieler/340950/saison/2020/verein/0/liga/0/wettbewerb/GB1/pos/0/trainer_id/0/plus/1) to get the mappings)

In [7]:
with_improved_columns = improve_columns(with_renamed_columns)


### Create
- [x] Add surrogate keys `game_id`, `player_id`, `appearance_id`, `home_club_id`, `away_club_id`
- [x] Split `result` into `home_club_goals` and `away_club_goals`
- [x] Approximate appearance `season`

In [8]:

with_new_columns = add_new_columns(with_improved_columns)

### Filter
* Only season 2020 is complete on the current file, so we remove the rest
* To reduce the scope of this version of the data prep scritp, select only appearances from domestic competitions


In [9]:
with_filtered_appearances = filter_appearances(with_new_columns)

## Validate
Validate that the output dataframe contains consistent data. Two types of checks are performed.

### Value checks
- [x] Fields `red_cards`, `yellow_cards`, `own_goals`, `assists`, `goals` and `minutes_played` contain values within an expected range
- [x] Rows are unique on `player_id` + `date`
- [ ] `position` field is either one of the long form player positions from Transfermarkt

### Completeness checks
- [x] Number of teams per domestic competition must be exactly 20
- [ ] Each club must play 38 games per season on the domestic competition
- [ ] On each match, both clubs should have at least 11 appearances
- [ ] Similarly, each club must have at least 11 appearances per game


In [10]:
validations = [
    'assert_df_not_empty',
    'assert_minutes_played_gt_90',
    'assert_goals_in_range',
    'assert_assists_in_range',
    'assert_own_goals_in_range',
    'assert_yellow_cards_range',
    'assert_red_cards_range',
    'assert_unique_on_player_and_date',
    'assert_clubs_per_domestic_competition',
    # 'assert_games_per_season_per_club',
    'assert_appearances_per_match',
    'assert_appearances_per_club_per_game',
    'assert_appearances_freshness_is_less_than_one_week',
    'assert_goals_ne_assists',
    'assert_goals_ne_own_goals',
    'assert_yellow_cards_not_constant',
    'assert_red_cards_not_constant'
]
validate(with_filtered_appearances, validations)

Validation assert_appearances_per_club_per_game did not pass: 
Validation assert_appearances_freshness_is_less_than_one_week did not pass: 8


## Save

In [11]:
with_filtered_appearances.to_csv(
  prep_file,
  index=False
)